In [1]:
%load_ext sql

In [2]:
%sql postgresql://appdev@data/appdev

'Connected: appdev@appdev'

### 1. 
Using a single join and a grouping, write a query that allows you to find the forename, surname, driverid and the total number of wins for each driver in the f1db schema.

In [7]:
%%sql SELECT forename, surname, drivers.driverid, COUNT(*) as wins FROM drivers
JOIN results ON drivers.driverid = results.driverid
WHERE position = 1
GROUP BY drivers.driverid, forename, surname
ORDER BY wins DESC LIMIT 10;

10 rows affected.


forename,surname,driverid,wins
Michael,Schumacher,30,91
Lewis,Hamilton,1,60
Alain,Prost,117,51
Sebastian,Vettel,20,46
Ayrton,Senna,102,41
Fernando,Alonso,4,32
Nigel,Mansell,95,31
Jackie,Stewart,328,27
Niki,Lauda,182,25
Jim,Clark,373,25


### 2. 
Write a query that joins the tables results, constructors and drivers to show the amount of times a driver has driven a car from a constructor. In other words how many times one driver (for instance Schumacher) has driven a constructor (for instance Mercedes). Your table should have three fields: drivers.driverref, constructors.name and count(*) (showing the number of times the driver has driven the constructor).

Hint: You can group on more than one value

In [8]:
%%sql SELECT driverref, name, COUNT(*) FROM results
JOIN drivers USING(driverid)
JOIN constructors USING(constructorid)
GROUP BY driverref, name 
ORDER BY COUNT DESC LIMIT 10;

10 rows affected.


driverref,name,count
michael_schumacher,Ferrari,181
coulthard,McLaren,150
massa,Ferrari,140
button,McLaren,137
rosberg,Mercedes,136
hakkinen,McLaren,133
laffite,Ligier,132
webber,Red Bull,129
raikkonen,Ferrari,126
vettel,Red Bull,113


### 3.
Now we have found the most driven vehicles even for those who didn't finish the race. Extend the query from 2 by removing all tuples from the result where the status is not "Finished".

In [9]:
%%sql SELECT driverref, name, COUNT(*) FROM results
JOIN drivers USING(driverid)
JOIN constructors USING(constructorid)
JOIN status USING(statusid)
WHERE status LIKE 'Finished' GROUP BY driverref, name
ORDER BY count DESC LIMIT 10;

10 rows affected.


driverref,name,count
michael_schumacher,Ferrari,141
massa,Ferrari,112
rosberg,Mercedes,109
webber,Red Bull,100
vettel,Red Bull,97
raikkonen,Ferrari,92
alonso,Ferrari,87
button,McLaren,87
coulthard,McLaren,85
hamilton,Mercedes,85


### 4.
Create a third (and new) query that finds the amount of milliseconds spent in pitstop (see the pitstops relation) by each unique combinations of driverid and raceid.

Hint: use the sum aggregate function to find the total amount of milliseconds
Hint: i recommend making this into a view

In [48]:
%%sql CREATE OR REPLACE VIEW pitstop_time AS
SELECT driverid, raceid, SUM(milliseconds) FROM pitstops
GROUP BY driverid, raceid;

Done.


[]

**Selecting all from the view pitstop time created in the query above.

In [49]:
%sql SELECT * FROM pitstop_time ORDER BY sum LIMIT 10;

10 rows affected.


driverid,raceid,sum
154,909,18928
17,877,19066
4,877,19368
18,877,19455
155,877,19529
13,877,19585
18,871,19668
822,955,19732
10,877,19884
3,955,20058


### 5.
Time to put everything together. Use the query for 4 as a join subquery in query 3 (see slides on "a query within a query") to find the total amount of pitstop time, for each result in query 3

In [50]:
%%sql CREATE OR REPLACE VIEW total_pitstop_time AS SELECT driverref, name, SUM(pitstop_time.sum) FROM results
JOIN drivers USING(driverid)
JOIN constructors USING(constructorid)
JOIN status USING(statusid)
JOIN pitstop_time USING(driverid, raceid)
WHERE status LIKE 'Finished' GROUP BY driverref, name
ORDER BY sum ASC;

Done.


[]

**Selecting all from the view total pitstop time created in the query above.

In [51]:
%sql SELECT * FROM total_pitstop_time LIMIT 10;

10 rows affected.


driverref,name,sum
ambrosio,Lotus F1,21962
rosa,HRT,22272
kobayashi,Caterham,53752
rosa,Sauber,54327
ericsson,Caterham,60508
kovalainen,Lotus F1,61656
kovalainen,Lotus,65463
trulli,Lotus,66025
glock,Marussia,82876
rossi,Manor Marussia,90080


### 6.
As a last thing we want to find the driver with the least pistop time. However we want to control for the amount of races (see explanation below), so we need to include one last column: the average pitstop time per race. Your table should be sorted based on the ascending values of that average pitstop time.

Explanation: If we just count the pitstop time, we only get the total time he spend in a workshop, no matter how many races he participated in. If a driver participated in 100 races, he would have more pitstop time than a driver participating in one race!
Hint: Use the sum function to accumulate the total pistop time and divide that with the amount of races the driver has participated in

**This view gets the total pitstop time for each driverreff.

In [52]:
%%sql CREATE OR REPLACE VIEW total_pitstop_time AS
SELECT driverref, name, SUM(pitstop_time.sum) FROM results
JOIN drivers USING(driverid)
JOIN constructors USING(constructorid)
JOIN status USING(statusid)
JOIN pitstop_time USING(driverid, raceid)
GROUP BY driverref, name
ORDER BY sum;

Done.


[]

**This view gets the number of times a driver appers in the results.

In [53]:
%%sql CREATE OR REPLACE VIEW race_count AS
SELECT driverref, count(driverid) FROM results
JOIN drivers USING(driverid)
GROUP BY driverref
ORDER BY count;

Done.


[]

In [55]:
%%sql SELECT driverref, racecount.count AS numberofraces, sum(total_pitstop_time.sum/racecount.count) AS average
FROM total_pitstop_time
JOIN racecount USING(driverref)
GROUP BY driverref, racecount.count
ORDER BY average LIMIT 1;

1 rows affected.


driverref,numberofraces,average
heidfeld,184,3115.9402173913043478
